In [99]:
import requests
import lxml 
import numpy as np
import re, json
import sys
import urllib.request
from urllib.request import urlopen
from lxml import html
from elasticsearch import Elasticsearch
import logging

r = requests.get("http://height-weight-chart.com")
urls = []



In [100]:
#Goes through and gets ending part of url for each specific image
for line in r.iter_lines():
    m = re.search("[0-9]{3}-[0-9]{3}.html", str(line.decode('utf-8')))
    if m == None:
        x = 0 
    else:
        urls.append(m.group(0)) 

In [101]:
allUrls = urls

In [102]:
def findHeight(line):
    h = re.search("[0-9]{1,3}\scm", line)
    return h
    

In [103]:
def findWeight(line):
    w = re.search("[0-9]{1,3}\skg", line)
    return w

In [104]:
def findImage(line):
    m = re.search("largepic.*[0-9]{3}-[0-9]{3}.*.jpg", line)
    return m
    

In [105]:
data_list = []
save_urls = []
def findAll():
    for url in urls:
        q = requests.get("http://height-weight-chart.com/" + url) 
        save_urls.append(q.content)
        colorNotFound = True
        heightFound = False
        weightFound = False
        imageFound = False
        height = 0
        weight = 0
        #x = "h"
        for line in q:
            lineToUse = str(line.decode("ISO-8859-1"))
            #need a way to also get the next line
            #print(" ")
            #print(lineToUse)

            if(colorNotFound == True):
                z = re.search("color=#0000FF", lineToUse)
                if z != None:
                    colorNotFound = False
                    h = findHeight(lineToUse)
                    if h != None:
                        heightFound = True
                        height = str(h.group(0))
                        w = findWeight(lineToUse) # want to add next line to it
                        #x = lineToUse
                        if w != None:
                            weightFound = True
                            weight = str(w.group(0))
            else:  
                if(heightFound == False):
                    h = findHeight(lineToUse)
                    if h != None:
                        heightFound = True
                        height = str(h.group(0))
                        w = findWeight(lineToUse)
                        if w != None:
                            weightFound = True
                            weight = str(w.group(0))

                elif(heightFound == True and weightFound == False):
                    w = findWeight(lineToUse)
                    if w != None:
                        weight = str(w.group(0))
                        weightFound = True
                        m = findImage(lineToUse)
                        if m != None:
                            data = {'url': "http://height-weight-chart.com/" + url, 'image': str(m.group(0)),
                                'height': height, 'weight': weight}
                            #print(data)
                            data_list.append(data)
                            colorNotFound = True
                            heightFound = False
                            weightFound = False
                            imageFound = False
                elif(heightFound == True and weightFound == True and imageFound == False):
                        m = findImage(lineToUse)
                        if m != None:
                            data = {'url': "http://height-weight-chart.com/" + url, 'image': str(m.group(0)),
                                'height': height, 'weight': weight}
                            data_list.append(data)
                            colorNotFound = True
                            heightFound = False
                            weightFound = False
                            imageFound = False
    

In [106]:
findAll()

In [108]:
print(data_list)

[{'url': 'http://height-weight-chart.com/608-180.html', 'image': 'largepic" src="l/608-180_Dan_L1.jpg', 'height': '203 cm', 'weight': '82 kg'}, {'url': 'http://height-weight-chart.com/611-490.html', 'image': 'largepic" src="l/611-490_Xavier_L1.jpg', 'height': '211 cm', 'weight': '222 kg'}, {'url': 'http://height-weight-chart.com/607-170.html', 'image': 'largepic" src="l/607-170_Lincoln_L1.jpg', 'height': '201 cm', 'weight': '77 kg'}, {'url': 'http://height-weight-chart.com/607-210.html', 'image': 'largepic" src="l/607-210_Allan_L1.jpg', 'height': '201 cm', 'weight': '95 kg'}, {'url': 'http://height-weight-chart.com/606-240.html', 'image': 'largepic" src="l/606-240_Kenny_L1.jpg', 'height': '198 cm', 'weight': '109 kg'}, {'url': 'http://height-weight-chart.com/606-250.html', 'image': 'largepic" src="l/606-250_Dale_L1.jpg', 'height': '198 cm', 'weight': '113 kg'}, {'url': 'http://height-weight-chart.com/606-260.html', 'image': 'largepic" src="l/606-260_Cees_L1.jpg', 'height': '198 cm', 'w

In [109]:
#go through array of objects and clean up data
import json
for x in range(0, len(data_list)):

    h = re.search("[0-9]{1,3}\s*cm", data_list[x]['height'])

    data_list[x]['height'] = {
      'amount': str(h.group(0)).split()[0],
      'unit': str(h.group(0)).split()[1]
    }
    
    w = re.search("[0-9]{1,3}\skg", data_list[x]['weight'])
    data_list[x]['weight'] = {
        'amount': str(w.group(0)).split()[0],
        'unit': str(w.group(0)).split()[1]
    }
    
    
    i = re.search("[0-9]{3}-[0-9]{3}.*.jpg", data_list[x]['image'])
    data_list[x]['image'] = "http://height-weight-chart.com/l/" + str(i.group(0))

print(json.dumps(data_list, indent=4, sort_keys=True))



[
    {
        "height": {
            "amount": "203",
            "unit": "cm"
        },
        "image": "http://height-weight-chart.com/l/608-180_Dan_L1.jpg",
        "url": "http://height-weight-chart.com/608-180.html",
        "weight": {
            "amount": "82",
            "unit": "kg"
        }
    },
    {
        "height": {
            "amount": "211",
            "unit": "cm"
        },
        "image": "http://height-weight-chart.com/l/611-490_Xavier_L1.jpg",
        "url": "http://height-weight-chart.com/611-490.html",
        "weight": {
            "amount": "222",
            "unit": "kg"
        }
    },
    {
        "height": {
            "amount": "201",
            "unit": "cm"
        },
        "image": "http://height-weight-chart.com/l/607-170_Lincoln_L1.jpg",
        "url": "http://height-weight-chart.com/607-170.html",
        "weight": {
            "amount": "77",
            "unit": "kg"
        }
    },
    {
        "height": {
            "amount

In [110]:
def calcBMI(height, weight):
    height = height / 100
    bmi = weight/(height * height)
    return bmi

In [116]:
print(len(data_list))

216


In [121]:
import urllib
import urllib.request
from urllib.request import urlretrieve
from PIL import Image
import random
import cv2
def imagesToFolder():
    count = 0
    for x in range(0, len(data_list)):
        try:
            height = int(data_list[x]['height']['amount'])
            weight = int(data_list[x]['weight']['amount'])
            if(len(str(weight)) > 1): 
                count = count + 1
                bmiNum = calcBMI(height, weight)
                url = data_list[x]['image']
                print(weight)
                nameOfImage = "B-"+ str(bmiNum) + "H-" + str(height) + "W-" + str(weight) + "-" + str(hash(random.random())) + "6_8.jpg"
                path = "/Users/nikki-genlife/Desktop/anaconda/all_usable_imageNames"
                response = urllib.request.urlopen(url)
                y = response.read()
                fd = open(path + "/" + nameOfImage, "wb")
                fd.write(y)
                fd.close()
            else:
                pass
        except urllib.error.HTTPError:
            pass
    print(count)

82
222
77
95
109
113
118
168
64
82
100
104
122
127
136
95
100
104
118
145
236
91
100
104
141
64
77
100
113
132
218
64
77
86
113
77
91
100
104
127
136
141
145
150
50
54
59
73
82
86
91
95
104
118
122
127
132
141
163
168
54
59
64
68
73
86
91
122
127
45
50
54
59
64
73
77
82
86
91
100
104
122
136
141
45
50
54
59
64
68
82
100
104
113
132
150
177
54
59
64
68
73
77
82
91
95
100
118
141
145
172
41
45
50
54
59
68
73
82
86
91
95
100
109
118
122
41
45
59
73
77
86
91
95
109
113
122
136
45
50
54
59
64
68
73
82
86
91
95
104
109
41
45
50
68
86
91
113
122
136
168
45
54
59
100
113
132
150
45
68
73
77
95
41
45
50
54
64
82
95
100
45
68
77
104
41
45
50
73
77
86
191
